In [5]:
from __future__ import division
import pandas as pd
import numpy as np 
import matplotlib.pylab as plt
from code.organize import *
from code.roughEDA import *
from code.survey_processor_full import *
from code.model_vis import *

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix,roc_curve
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA

from treeinterpreter import treeinterpreter as ti
import matplotlib.cm as cm
import cPickle as pickle
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Feature Contribution Analysis

In this session, how does each feature contribute to random forest decision making will be analysed. 
## 1. Load feature contribution matrix and group by cluster id

In [7]:
fc_df = pd.read_pickle('data/fc_df.pkl')

In [23]:
summary = fc_df.groupby('cluster')['employed'].agg([np.mean,np.size])
summary.columns = ['employment_rate','sample_size']
summary

,employment_rate,sample_size
cluster,,
0,0.058565,683
1,0.927550,2098
2,0.077778,360
3,0.067839,398
4,0.895210,668
5,0.889313,262


**Cluster 0,2,3** are unemployed groups. **Cluster 1,4,5** are employed groups. 

Let's look at unemployed group first. 

### Cluster 1: Unemployed

In [46]:
c1 = fc_df[fc_df.cluster == 0]
c1.drop(['employed','correct'],axis = 1,inplace=True)

/Users/yleng/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [52]:
sorted(zip(c1.mean(axis =0),c1.columns),key = lambda x: abs(x[0]),reverse = True)[:10]

[(-0.037370748829345088, 'age'),
 (-0.035867872897539836, 'educom'),
 (-0.032106094296888495, 'sphrs1'),
 (-0.024519725370472405, 'agekdbrn'),
 (-0.021255684375609853, 'divorce'),
 (-0.019506785944929821, 'partyid'),
 (-0.017024391535317502, 'polviews'),
 (-0.016792892486347143, 'res16'),
 (-0.01645438550821406, 'incom16'),
 (-0.013795521161037029, 'reliten')]